In [ ]:
import data_processor_multi_processing as processor
from data_service import DataService

In [2]:
ds = DataService("ETH-USDT", "3min")
duplicates = ds.remove_duplicate_collections()



KeyboardInterrupt: 

In [ ]:
collections = ds.get_all_collection_names()
display(collections)

In [ ]:
symbols = ['ETH-USDT', 'BTC-USDT']
timeframe = '1min'
main_symbol = 'ETH-USDT'
trade_type = 'long'
profit_target = 0.01
collection_name = 'example_collection'
processor.process_and_save_market_data(symbols, timeframe, main_symbol, trade_type, profit_target, collection_name)